In [1]:
import os 
import glob
import pandas as pd
from ics import Calendar, Event
from dataclasses import dataclass, field 
from collections import defaultdict
from typing import List, Dict
import numpy as np
from datetime import datetime

In [2]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [12]:
base_folder = os.getcwd()
source_folder = base_folder + '/healthlake/'
apple_health_files = glob.glob(source_folder + '*.json')


df_raw = pd.DataFrame()

import json
with open('config/event_types.json', 'r') as f:
    EVENT_TYPES = json.load(f)
    
with open('config/units.json', 'r') as f: 
    UNITS = json.load(f)

cols = UNITS.get('cols')
names = UNITS.get('names')

COLUMN_EVENT_TYPES = dict((col_name, types) for types, cols in EVENT_TYPES.items() for col_name in cols)


# read in raw data 
for json_file in apple_health_files:
    json_raw = pd.read_json(json_file, lines = True)
    df_raw = pd.concat([df_raw, json_raw])

In [4]:
def create_day_events(stats: pd.DataFrame, event_date: str): 
    """
    Iterate through different event types (food / activity / sleep)
    and generate events to add to the daily calendar only if event exists
    """
    day_events = []
    for types, col_names in EVENT_TYPES.items(): 
        # col_names = type_values['col_names']
        # obj_args = type_values['obj_args']
        
        print(types, col_names)
        # collect object name and arguments
        # dynamically create event type objects 
        dataclass_name = types.capitalize()
        dataclass_obj = globals()[dataclass_name]
        
        # collect object arguments to initialise objects from stats
        obj_args = stats[stats['name'].isin(col_names)][['name', 'qty']]
        obj_args = dict(obj_args.values)
        print(obj_args)
        
        
        if obj_args: 
            obj = dataclass_obj(**obj_args)
            print(obj)
            e = AppleHealthEvent(
                date = event_date,
                title = obj.title, 
                description = obj.description
            ).event
            day_events.append(e)
            
    return day_events


In [5]:
@dataclass 
class AppleHealthEvent(Event):
    """
    An event derived from Apple Health data
    For usage within .ics format 
    """
    date: datetime.date
    description: str 
    title: str
            
    @property 
    def event(self):
        all_day_date = f"{self.date} 00:00:00"
        e = Event()
        e.name = self.title
        e.description = self.description
        e.begin = all_day_date
        e.end = all_day_date
        e.make_all_day()

        return e

In [64]:
from typing import Optional
@dataclass
class Time: 
    "A basic time object in hours"
    time: Optional[float] = field(default=0)
    timeInMinutes: Optional[float] = field(default=0)
        
    def __post_init(self): 
        if self.timeInMinutes:
            self.time = self.timeInMinutes / 60
            
        
    @property 
    def hours(self) -> float: 
        hours = int(self.time)
        return hours 
    
    @property
    def minutes(self) -> str:
        minutes = int((self.time - self.hours) * 60)
        return minutes
    
    @property
    def title(self) -> str:
        title = f"{self.hours}h {self.minutes}m"
        return title

@dataclass 
class Food: 
    "A basic food object"
    carbohydrates: float 
    protein: float 
    total_fat: float 
    fiber: float 

    def __post_init__(self): 
        # rename objects for easier usage 
        self.carb = self.carbohydrates
        self.fat = self.total_fat
    
    @property 
    def calories(self) -> float: 
        calories = (self.carb + self.protein) * 4 + (self.fat) * 9
        return calories
    
    @property 
    def macros(self) -> str: 
        return f"{self.carb:.0f}C, {self.protein:.0f}P, {self.fat:.0f}F"

    @property 
    def title(self) -> str: 
        title = f"🔥 {self.calories:.0f} cals ({self.macros})" 
        return title 
    
    @property 
    def description(self) -> str: 
        description = f"""
        🔥 {self.calories:.0f} kcal
        🥞 {self.macros} 
        🍇 {self.fiber:.0f}
        """
        return description

@dataclass
class Sleep:
    "A basic sleep object"
    asleep: Time
    inBed: Time
    inBedStart: str

    def __post_init__(self): 
        # rename objects for easier usage
        self.time_asleep = self.asleep
        self.time_in_bed = self.inBed
        self.in_bed_time = self.inBedStart
    
    @property 
    def efficiency(self) -> float: 
        efficiency = self.time_asleep.time / self.time_in_bed.time * 100
        return efficiency
    
    @property
    def efficiency_title(self) -> str: 
        efficient = f"{self.efficiency:.0f}%"
        efficiency_title = f"🛏️ {efficient}"
        return efficiency_title
    
    @property 
    def title(self) -> str: 
        title = f"💤 {self.time_asleep.title} ({self.in_bed_time})"
        return title 
    
    @property
    def description(self) -> str: 
        description = f"""
        💤 Time asleep: {self.time_asleep.title}
        🛏️ Time in bed: {self.time_in_bed.title}
        🧮 Efficiency: {self.efficiency_title}
        """
        return description


In [65]:
@dataclass
class Activity: 
    "A basic activity for activity and mindfulness"
    apple_exercise_time: Time
    mindful_minutes: Time = None
    
    def __post_init__(self): 
        # rename objects for easier usage
        self.apple_exercise_time = Time(timeInMinutes=self.apple_exercise_time)
        self.mindful_minutes = Time(timeInMinutes=self.mindful_minutes)
    
    @property 
    def activity_description(self) -> str: 
        a_description = f"🚴‍♂️ Activity: {self.apple_exercise_time.title} active" 
        return a_description

    @property 
    def mindful_description(self) -> str:
        m_description = f"🧘 Mindful: {self.mindful_minutes.title} mindful"
        return m_description
    
    @property 
    def mindful_title(self) -> str: 
        block = np.floor(self.mindful_minutes.minutes / 10)
        title = f"{block}"
        return title
  
    @property 
    def title(self) -> str: 
        title = f"🧠 {self.mindful_title}"
        return title 
        
    @property 
    def description(self) -> str: 
        description = f"""
        {self.activity_description}
        {self.mindful_description}
        """
        print(description)
        return description

In [66]:
c = Calendar()
available_dates = ahc['dates'].unique()

for date in available_dates:
    daily_stats = ahc[ahc['dates'] == date]            
    daily_calendar = create_day_events(
        stats=daily_stats,
        event_date=date
    )
    for event in daily_calendar:
        c.events.add(event)

food ['carbohydrates', 'protein', 'total_fat', 'fiber']
{'carbohydrates': 184.14603999725, 'fiber': 26.6000000018, 'protein': 184.58643333115003, 'total_fat': 51.16325999105}
Food(carbohydrates=184.14603999725, protein=184.58643333115003, total_fat=51.16325999105, fiber=26.6000000018)
activity ['apple_exercise_time', 'mindful_minutes']
{'apple_exercise_time': 66.0}
Activity(apple_exercise_time=Time(time=0, timeInMinutes=66.0), mindful_minutes=Time(time=0, timeInMinutes=None))

        🚴‍♂️ Activity: 0h 0m active
        🧘 Mindful: 0h 0m mindful
        
sleep ['asleep', 'inBed', 'inBedStart']
{}
food ['carbohydrates', 'protein', 'total_fat', 'fiber']
{'carbohydrates': 195.81060606999998, 'fiber': 28.465151506399998, 'protein': 187.6671122999, 'total_fat': 64.322504458}
Food(carbohydrates=195.81060606999998, protein=187.6671122999, total_fat=64.322504458, fiber=28.465151506399998)
activity ['apple_exercise_time', 'mindful_minutes']
{'apple_exercise_time': 131.0}
Activity(apple_exercise_

In [55]:
ics_file_name = 'outputs/apple-health.ics'
with open(ics_file_name, 'w') as f:
    f.write(str(c))
    f.close()

/Users/ntonthat/Library/Caches/pypoetry/virtualenvs/apple-health-calendar-FpPaTjHZ-py3.11/lib/python3.11/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


In [9]:
def convert_kj_to_cal(row, new_name):
    """Converts kj to calories"""
    row_dict = row.to_dict()
    calorie_value = int(row['qty']/4)
    row_dict['qty'] = calorie_value
    row_dict['name'] = new_name
    row_dict['units'] = 'kcal'
    
    return pd.DataFrame(row_dict, index=[0])


def create_events(df):
    c = Calendar()
    available_dates = ahc['dates'].unique()
    
    for date in available_dates:
        daily_stats = ahc[ahc['dates'] == date]            
        daily_calendar = create_day_calendar(
            stats=daily_stats,
            event_date = date
        )
        print(daily_calendar)
        c.events.add(daily_calendar)

In [14]:
# config driven
sleep_stats_cols = ['asleep', 'inBed', 'inBedStart']
# ETL transformations 

df_ahc = df_raw.copy() 

# define transformations to go from df_raw to df_ahc (apple-health-calendar)
# cleaning values 
df_ahc['dates'] = pd.to_datetime(df_ahc['date']).dt.date

# create sleep stats
sleep_stats_cols.insert(0, 'dates')

daily_sleep = df_ahc.query('name == "sleep_analysis" \
    and sleepSource == "AutoSleep"')[sleep_stats_cols]

daily_sleep['inBedStart'] = pd.to_datetime(daily_sleep['inBedStart']).dt.time
daily_sleep['name'] = 'sleep'

# # create calories 
active_energy_rows = df_ahc[df_ahc['name'] == 'active_energy'][cols]
dietary_energy_rows = df_ahc[df_ahc['name'] == 'dietary_energy'][cols]

for _, row in active_energy_rows.iterrows():
    df_row = convert_kj_to_cal(row, 'calories_burnt')    
    df_ahc = pd.concat([df_ahc, df_row])
    
for _, row in dietary_energy_rows.iterrows(): 
    df_row = convert_kj_to_cal(row, 'calories_consumed')
    df_ahc = pd.concat([df_ahc, df_row])

def create_type_column(df: pd.DataFrame): 
    for _, row in df.iterrows(): 
        col_name = row['name']
        event_type = COLUMN_EVENT_TYPES.get(col_name)
        df['type'] = event_type
    return df

ahc = create_type_column(df_ahc)
    
# filter out values 

# for _, row in df_ahc.iterrows(): 
#     col_name = row['name']
#     event_type = COLUMN_EVENT_TYPES.get(col_name)
#     df_ahc['type'] = event_type


# ahc = df_ahc[~df_ahc['type'].isna()].copy()[cols]
# ahc = ahc.reset_index(drop=True)
# ahc['qty']  =ahc['qty'].round(2)


# # round values 
# df_ahc['qty'] = df_ahc['qty'].round(2)


In [15]:
ahc

,qty,date,name,units,Avg,Min,Max,sleepStart,sleepEnd,inBedEnd,sleepSource,asleep,inBed,inBedStart,inBedSource,dates,type
0,3039.918672,2022-09-10 00:00:00+09:30,active_energy,kJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-10,None
1,3553.969096,2022-09-11 00:00:00+09:30,active_energy,kJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-11,None
2,3627.758120,2022-09-12 00:00:00+09:30,active_energy,kJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,None
3,3750.788640,2022-09-13 00:00:00+09:30,active_energy,kJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-13,None
4,4264.282592,2022-09-14 00:00:00+09:30,active_energy,kJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-14,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2012.000000,NaT,calories_consumed,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,None
0,2057.000000,NaT,calories_consumed,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-13,None
0,2726.000000,NaT,calories_consumed,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-14,None
0,2665.000000,NaT,calories_consumed,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-15,None


In [16]:
df_ahc[df_ahc['name'] == 'carbohydrates']

,qty,date,name,units,Avg,Min,Max,sleepStart,sleepEnd,inBedEnd,sleepSource,asleep,inBed,inBedStart,inBedSource,dates,type
49,184.146040,2022-09-10 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-10,None
50,195.810606,2022-09-11 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-11,None
51,224.112857,2022-09-12 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-12,None
52,213.875907,2022-09-13 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-13,None
53,373.249320,2022-09-14 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-14,None
54,280.213590,2022-09-15 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-15,None
55,265.301367,2022-09-16 00:00:00+09:30,carbohydrates,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-16,None


In [17]:
def generate_calendar(df):
    """
    Generates a CSV and ICS from the dataframe
    :param df: cleansed dataframe from `create_description_cols`
    :param outputs: as type string - a combination of both the local and public storage
    """

    # output_csv_path = f"{output_path}/{file_name}.csv"
    # calendar_file_name = f'{file_name}.ics'
    file_name = 'apple_health'

    csv_file_name = f"{file_name}.csv"
    ics_file_name  = f"{file_name}.ics"


    LOGGER.info("Generating calendar (as .ICS)")
    c = Calendar()
    for _, row in df.iterrows():
        e = create_event(row['date'], row['name'], row['dsc'])
        c.events.add(e)

    df.to_csv(csv_file_name, index=False)

    with open(ics_file_name, 'w') as f:
        f.write(str(c))
        f.close()

    LOGGER.info("Outputing CSV and ICS to: %s", csv_file_name)
    return

In [23]:
ahc[ahc['dates'] == date] 

TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object

In [133]:
c = Calendar()
available_dates = ahc['dates'].unique()

for date in available_dates:
    daily_stats = ahc[ahc['dates'] == date]            
    daily_events = create_day_events(
        stats=daily_stats,
        event_date=date
    )

              name         qty
49   carbohydrates  184.146040
96           fiber   26.600000
194        protein  184.586433
275      total_fat   51.163260
Empty DataFrame
Columns: [name, qty]
Index: []
Empty DataFrame
Columns: [name, qty]
Index: []
              name         qty
50   carbohydrates  195.810606
97           fiber   28.465152
195        protein  187.667112
276      total_fat   64.322504
Empty DataFrame
Columns: [name, qty]
Index: []
Empty DataFrame
Columns: [name, qty]
Index: []
              name         qty
51   carbohydrates  224.112857
98           fiber   65.828571
196        protein  179.605714
277      total_fat   79.277857
Empty DataFrame
Columns: [name, qty]
Index: []
Empty DataFrame
Columns: [name, qty]
Index: []
              name         qty
52   carbohydrates  213.875907
99           fiber   46.463571
197        protein  167.182214
278      total_fat   46.543307
Empty DataFrame
Columns: [name, qty]
Index: []
Empty DataFrame
Columns: [name, qty]
Index: []
    

In [117]:
with open('apple_health.ics', 'w') as f:
    f.write(str(c))
    f.close()